In [1]:
import numpy
from pyspark.sql import SQLContext
import mysql.connector
from mysql.connector import errorcode

init parameter

In [2]:
param={}


#crate table
TABLES={}
TABLES['info']=(
    "CREATE TABLE IF NOT EXISTS info("
    "`id` INTEGER,"
    "`name` varchar(10),"
    "`path` varchar(256),"
    " PRIMARY KEY (`id`)"
    ")")
DB_NAME='aibgraz'
cnx=mysql.connector.connect(user='sh',host='localhost')
cursor=cnx.cursor()


try:
    cnx.database = DB_NAME
except mysql.connector.Error as err:
    print(err)
    exit(1)
    

for name, dll in TABLES.iteritems():
    cursor.execute(dll)

cursor.close()
cnx.close()

########################################
#load image databases, and information
########################################

In [3]:
sqlContext=SQLContext(sc)
df_graz02 = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://127.0.0.1:3306',
    dbtable='imgBase.graz_02',
    driver='com.mysql.jdbc.Driver',
    user='sh').load()

df_graz02_info = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://127.0.0.1:3306',
    dbtable='imgBase.graz_02_info',
    driver='com.mysql.jdbc.Driver',
    user='sh').load()
#type(df_graz02)
#print(df_graz02.count())

rdd_graz02 = df_graz02.rdd

bag of feature image categorization

select category/training/testing

In [4]:
def sel(r):
    if (r.category=='person') and (r.cat_ind<=10):
        return True
    else:
        return False
    
rdd_graz02_select = rdd_graz02.filter(sel);
rdd_graz02_select.first()

Row(ind=1166, cat_ind=1, name=u'person_001', category=u'person', path=u'/home/sh/Data/TestData/Graz-02/data/person/person_001.bmp', groundtruth_path=u'/home/sh/Data/TestData/Graz-02/groundtruth/person/person_001_gt.jpg')

###############################
#feature extraction
##############################

In [5]:
# load image from Row.path,
# extract feature of that image
# and save into file, whose path is Row.

import cv2
from scipy.misc import imread


def block_feat(Row):
    img = imread(Row.path)
    featureDetector=cv2.xfeatures2d.SIFT_create()
    descExtractor = featureDetector
    
    kp, des = featureDetector.detectAndCompute(img, None)
    #print(dir(kp))
    #img2=cv2.drawKeypoints(img, kp)
    #cv2.imwrite('sift_keypoint.jpg', img2)
    return des


rdd_feat = rdd_graz02_select.map(block_feat)
print(rdd_feat.count())
#print(rdd_feat)
#print(rdd_feat.first())

10


train dictionary
kmeans